In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import matplotlib.pyplot as plt
import tensorflow as tf #numpy needs to be 1.23 or earlier in order for tf to import correctly


In [2]:
#make the spark dataframe df a pandas dataframe
df = pd.read_csv("income.csv")

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23149 entries, 0 to 23148
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             23149 non-null  int64 
 1   workclass       23149 non-null  object
 2   education       23149 non-null  object
 3   marital_status  23149 non-null  object
 4   occupation      23149 non-null  object
 5   relationship    23149 non-null  object
 6   race            23149 non-null  object
 7   sex             23149 non-null  object
 8   hours_per_week  23149 non-null  int64 
 9   native_country  23149 non-null  object
 10  income          23149 non-null  object
dtypes: int64(2), object(9)
memory usage: 1.9+ MB


In [4]:
df_num = df[["age", "hours_per_week"]]
df_obj = df.drop(["age", "hours_per_week"], axis = 1)
df_obj = df_obj.astype(str)
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse_output=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df_obj))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out()

# Merge one-hot encoded features and drop the originals
df = df_num.merge(encode_df,left_index=True, right_index=True)


In [5]:
#pull out our variables
y= df[["income_>50K"]]
X= df.drop(['income_<=50K', 'income_>50K'], axis = 1)

# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X)

# Scale the data
X_scaled = X_scaler.transform(X)

In [6]:
# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, random_state=78)

In [7]:
# Define the model - deep neural net
#optimization attempt - using more layers
number_input_features = len(X_train[0])
hidden_nodes_layer1 = 25
hidden_nodes_layer2 = 12
hidden_nodes_layer3 = 6


nn = tf.keras.models.Sequential()

# First hidden layer
#Optimization attempt - added a layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 25)                2500      
                                                                 
 dense_1 (Dense)             (None, 12)                312       
                                                                 
 dense_2 (Dense)             (None, 6)                 78        
                                                                 
 dense_3 (Dense)             (None, 1)                 7         
                                                                 
Total params: 2897 (11.32 KB)
Trainable params: 2897 (11.32 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [8]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [9]:
# Train the model
#Optimization attempt - increased the epochs from 25 to 50
fit_model = nn.fit(X_train,y_train,epochs=50)

Epoch 1/50
543/543 [==============================] - 1s 981us/step - loss: 0.4161 - accuracy: 0.7925
Epoch 2/50
543/543 [==============================] - 1s 1ms/step - loss: 0.3624 - accuracy: 0.8268
Epoch 3/50
543/543 [==============================] - 1s 993us/step - loss: 0.3529 - accuracy: 0.8294
Epoch 4/50
543/543 [==============================] - 0s 903us/step - loss: 0.3483 - accuracy: 0.8331
Epoch 5/50
543/543 [==============================] - 0s 882us/step - loss: 0.3440 - accuracy: 0.8356
Epoch 6/50
543/543 [==============================] - 1s 977us/step - loss: 0.3410 - accuracy: 0.8378
Epoch 7/50
543/543 [==============================] - 1s 950us/step - loss: 0.3392 - accuracy: 0.8390
Epoch 8/50
543/543 [==============================] - 1s 917us/step - loss: 0.3359 - accuracy: 0.8403
Epoch 9/50
543/543 [==============================] - 0s 902us/step - loss: 0.3347 - accuracy: 0.8398
Epoch 10/50
543/543 [==============================] - 0s 906us/step - loss: 0.3324 

In [10]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test,y_test,verbose=2)


181/181 - 0s - loss: 0.4063 - accuracy: 0.8272 - 252ms/epoch - 1ms/step


In [11]:

print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

Loss: 0.40628352761268616, Accuracy: 0.8272287249565125


In [12]:
import pickle

pickle.dump(nn, open('model.pkl', 'wb'))